# 데이터 입력

In [3]:
import sys
from dotenv import load_dotenv # 입력 파트 종합본에 빼둘 것.
load_dotenv(override=True)

max_invalid = 4
invalid_count = 0

while True:
    choice = input("입력데이터 선택\n 1. 데이터셋  2. 논문\n숫자(1 또는 2)를 입력하세요: ").strip()

    if choice == "1":
        # 데이터셋 호출 API
        svc_id = input("데이터셋 id :") # b37f0c9413eeb7c45f6fe31cbe3a41ef
        print(f"데이터셋을 호출합니다...with ID: {svc_id}")
        data = Meta_API_call(svc_id)
        print(type(data)) # type: dict
        break
    elif choice == "2":
        # 논문 호출 API
        print("논문을 호출합니다...")
        pass # 논문  호출 API
        break
    else:
        invalid_count += 1
        remaining = max_invalid - invalid_count
        if remaining <= 0:
            print("입력 오류가 너무 많아 프로그램을 종료합니다.")
            sys.exit(1)
        else:
            print(f"잘못된 입력입니다. {remaining}번 남았습니다. 다시 입력해주세요.")

데이터셋을 호출합니다...with ID: b37f0c9413eeb7c45f6fe31cbe3a41ef


NameError: name 'Meta_API_call' is not defined

## 연구데이터 메타정보 조회 API(함수)

In [2]:
import os, requests
from dotenv import load_dotenv
load_dotenv(override=True)

def DATA_browse(API_KEY, svc_id):
    API_KEY = os.getenv("DATAON_META_API_KEY")
    assert API_KEY and API_KEY.strip(), "환경변수(DATAON_META_API_KEY)가 비어있어요!"

    url = "https://dataon.kisti.re.kr/rest/api/search/dataset/"+svc_id
    params = {"key": API_KEY}

    # print("🔎 호출 URL 미리보기:", requests.Request('GET', url, params=params).prepare().url)
    res = requests.get(url, params=params, timeout=20)
    print("HTTP", res.status_code)
    data = res.json()
    print(data)
    print(len(data['records']))
    return data

## 검색 노드(함수)

In [ ]:
import os, requests
from dotenv import load_dotenv

load_dotenv()
# API_KEY = os.getenv("DATAON_SEARCH_API_KEY")
# assert API_KEY and API_KEY.strip(), "환경변수(DATAON_API_KEY)가 비어있어요!"

# url = "https://dataon.kisti.re.kr/rest/api/search/dataset/"
# params = {"key": API_KEY, "query": "korean text", "from": 0, "size": 5}
# # key / CHAR / 필수 / API_KEY
# # query / CHAR / 필수 / 검색키워드
# # from / CHAR / 옵션 / 페이지시작위치
# # size / CHAR / 옵션 / 페이지사이즈

# print("🔎 호출 URL 미리보기:", requests.Request('GET', url, params=params).prepare().url)
# res = requests.get(url, params=params, timeout=20)
# print("HTTP", res.status_code)
# data = res.json()
# print(data)


def Search_API_Call(query: str):
    API_KEY = os.getenv("DATAON_SEARCH_API_KEY")
    assert API_KEY and API_KEY.strip(), "환경변수(DATAON_API_KEY)가 비어있어요!"

    url = "https://dataon.kisti.re.kr/rest/api/search/dataset/"
    params = {"key": API_KEY, "query": "korean text", "from": 0, "size": 5}
    # key / CHAR / 필수 / API_KEY
    # query / CHAR / 필수 / 검색키워드
    # from / CHAR / 옵션 / 페이지시작위치
    # size / CHAR / 옵션 / 페이지사이즈

    # print("🔎 호출 URL 미리보기:", requests.Request('GET', url, params=params).prepare().url)
    res = requests.get(url, params=params, timeout=20)
    print("HTTP", res.status_code)
    data = res.json()
    print(data)

🔎 호출 URL 미리보기: https://dataon.kisti.re.kr/rest/api/search/dataset/?key=D62EA22E8D9BBAFEFA15D7A0F39FD79F&query=korean+text&from=0&size=5
HTTP 200
{'response': {'elapsed time': '79 ms', 'status': '200', 'message': 'OK', 'total count': 28, 'type': 'json', 'key': 'D62EA22E8D9BBAFEFA15D7A0F39FD79F', 'query': 'korean text', 'from': 0, 'size': 5}, 'records': [{'svc_id': '2a8dadd5a1fb22affc1db8a345d66de4', 'ctlg_type': '02', 'dataset_type': '02', 'ctlg_type_pc': 'dataset', 'dataset_type_pc': '해외', 'dataset_pub_dt_pc': '2024', 'dataset_access_type_pc': '공개', 'file_yn_pc': '랜딩페이지이동', 'dataset_cc_license_pc': 'none', 'dataset_main_lang_pc': 'English', 'dataset_sub_lang_pc': 'none', 'cltfm_pc': 'ARDC', 'dataset_title_kor': '', 'dataset_title_etc_main': 'Weak Force 3', 'dataset_title_etc_sub': '', 'dataset_expl_kor': '', 'dataset_expl_etc_main': 'Research background: There has been a significant rise in group-based and collective art practice that challenges conventional methodologies and ideas of 

In [51]:
# resp 는 이미 requests.get(...) 결과라고 가정
import json
import re

print("HTTP", resp.status_code)

# 1) JSON 파싱 시도 — pretty print
try:
    data = resp.json()
    print("response JSON:")
    print(json.dumps(data, ensure_ascii=False, indent=2))

    # 2) 토큰 추출 시도(자주 쓰이는 키 목록 검사)
    candidate_keys = ["access_token", "accessToken", "token", "accessTokenInfo", "result"]
    token = None
    for k in candidate_keys:
        # 직접 키가 있는 경우
        if isinstance(data, dict) and k in data:
            # 직접 문자열이면 사용, dict이면 더 찾아봄
            v = data[k]
            if isinstance(v, str):
                token = v; break
            if isinstance(v, dict):
                # 내부에 access_token 있을 수도 있음
                for subk in ("access_token", "accessToken", "token"):
                    if subk in v and isinstance(v[subk], str):
                        token = v[subk]; break
                if token: break

    if token:
        print("access token (found):", token)
    else:
        print("access token 키를 자동으로 찾지 못함. 위 JSON을 확인해줘.")
except ValueError:
    # JSON 아님 -> 텍스트 검색 시도
    text = resp.text
    print("response text (non-json):")
    print(text[:1000])  # 너무 길면 일부만 출력

    # 간단한 토큰 패턴(예: 길고 랜덤한 문자열) 탐색(선택적)
    m = re.search(r'([A-Za-z0-9\-\._]{20,200})', text)
    if m:
        print("heuristic token-like match:", m.group(1))
    else:
        print("토큰 형태의 문자열을 찾지 못함. 전체 응답을 확인해봐.")

HTTP 200
response JSON:
{
  "errorMessage": "(토큰발급용)MAC Address 확인 불가",
  "errorCode": "E4006",
  "statusMessage": "Bad Request",
  "statusCode": "400"
}
access token 키를 자동으로 찾지 못함. 위 JSON을 확인해줘.


In [48]:
from datetime import datetime
now = datetime.now().strftime('%Y%m%d%H%M%S')
print(type(now))


<class 'str'>


In [60]:
from dotenv import load_dotenv, find_dotenv, dotenv_values

load_dotenv(override=True)
MAC_ADDRESS = os.getenv('MAC_ADDRESS')
print(MAC_ADDRESS, dt)

6E-BD-E0-6B-BE-DE 20251007230040


디버깅 중

In [46]:
b64 = base64.b64encode(ciphertext).decode('ascii')        # ciphertext만
iv_b64 = base64.b64encode(iv).decode('ascii')
params = {"client_id": CLIENT_ID, "accounts": b64, "iv": iv_b64}
r = requests.get("https://apigateway.kisti.re.kr/tokenrequest.do", params=params, timeout=15)
print("iv-param test ->", r.status_code, r.url)
print(r.text[:300])

iv-param test -> 200 https://apigateway.kisti.re.kr/tokenrequest.do?client_id=e37c8354a854dad4e74fe1238b2921f8e47e34ddcaacbd3243d01caa93e874c8&accounts=4o7DiRqSx7BVacjy9RbhdrIV3%2FME%2F19wakmZOLfgBpWMYeu3HpiK%2BzoPKJCeJIDeqiq%2B2npi5pAVIQ0ZZnnUpw%3D%3D&iv=xmRmNXYcpODyH71uOwivlA%3D%3D
{"errorMessage":"(토큰발급용)MAC Address 확인 불가","errorCode":"E4006","statusMessage":"Bad Request","statusCode":"400"}


# 아래가 지금 쓸 예정인 것.

In [124]:
import os, requests, json, base64
from datetime import datetime
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from dotenv import load_dotenv
from urllib import parse

load_dotenv(override=True)
CLIENT_ID = os.getenv("SCIENCEON_CLIENT_ID")
API_KEY = os.getenv("SCIENCEON_API_KEY")  # 네가 발급받은 32자리 인증키(문자열)
MAC_RAW = os.getenv("MAC_ADDRESS")

# 1) 정규화 (필수)
mac = (MAC_RAW or "").strip().upper().replace(":", "-")
if not mac:
    raise SystemExit("MAC_ADDRESS가 비어있음")
print(mac)

# 2) datetime
dt = datetime.now().strftime('%Y%m%d%H%M%S')
print(dt)

# 3) accounts json (compact)
accounts_json = json.dumps({"mac_address": mac, "datetime": dt}, separators=(",", ":")).encode()
# print("accounts_json:", accounts_json.decode('utf-8'))
# print(mac, dt)

# 4) key 준비 (확인: key_bytes 길이 반드시 16/24/32)
key_bytes = bytes(API_KEY, 'utf-8')
print("len(key_bytes)=", len(key_bytes))

if len(key_bytes) not in (16,24,32):
    raise SystemExit("API_KEY 길이 문제: AES 키는 16/24/32 바이트여야 함")

# 5) 암호화 (ECB)
cipher = AES.new(key_bytes, AES.MODE_ECB)
ciphertext = cipher.encrypt(pad(accounts_json, AES.block_size))

result = parse.quote(ciphertext)
params = {"accounts": result, "client_id": CLIENT_ID}


# 8) 요청 (requests가 자동 URL 인코딩 함)
url = "https://apigateway.kisti.re.kr/tokenrequest.do"
r = requests.get(url, params=params, timeout=15)
print("URL sent:", r.url)
print("HTTP", r.status_code)
print(r.text[:800])

1C-57-DC-50-BB-31
20251008202421
len(key_bytes)= 32
URL sent: https://apigateway.kisti.re.kr/tokenrequest.do?accounts=%255BD%2523%25DC%2524Xv%25DE%25A4%25A8%25B3%25E0%2592%259D%25E0h%259B%2540%2584%253D%255B%25B3pR%25D4%2590%2520A%25FD.%25F3%25FD%2510%25FE%2580%25A2%25B4J%25C6%258F%2591%2505%2512%2595%2540%2512%253A%25E3%25E8%25AB%2500mvy%2521%25AD%2504%25A3%250E%25C1%250A4%25CB3&client_id=e37c8354a854dad4e74fe1238b2921f8e47e34ddcaacbd3243d01caa93e874c8
HTTP 200
{"errorMessage":"(토큰발급용)MAC Address 확인 불가","errorCode":"E4006","statusMessage":"Bad Request","statusCode":"400"}


In [96]:
key_bytes

b'7cd1621a60d946f4997329ecdfce1df3'

In [4]:
import base64, urllib.parse

# b64_std 는 네가 전송한 b64 문자열 (패딩 유지)
print("b64 preview:", b64_std[:80])
print("endswith '=' :", b64_std.endswith("="))

# 서버가 받을 실제 텍스트(예: requests가 전송하는 URL 이후에 서버가 디코딩할 값)를
# 시뮬레이션하려면 URL 인코딩 후 다시 디코딩해보면 된다:
encoded = urllib.parse.quote_plus(b64_std)   # requests가 URL에 넣는 형태
decoded = urllib.parse.unquote_plus(encoded)
assert decoded == b64_std
# base64 디코딩으로 payload가 돌아오는지 확인
payload = base64.b64decode(decoded)

b64 preview: W0Qj3CRYdt6kqLPgkp3gaJtAhD1bs3BS1JAgQf0u8/0Q/oCitErGj5EFEpVAEjrjUDxnvlQYt1CXHjnm
endswith '=' : True


In [ ]:
t = {"mac_address": mac, "datetime": dt}
tj = json.dumps(t)
type(tj)
tj

# 2) 파일로 저장 (추천: json.dump, 사람이 읽기 좋게 indent 추가)
with open("data.json", "w", encoding="utf-8") as f:
    json.dump(t, f, ensure_ascii=False, indent=2)

# 3) 파일에서 읽기
with open("data.json", "r", encoding="utf-8") as f:
    data = json.load(f)
print(type(data))  # <class 'dict'>
print(data)   


<class 'dict'>
{'mac_address': '1C-57-DC-50-BB-31', 'datetime': '20251007234828'}


In [ ]:
# 1) accounts JSON 생성
accounts_json = make_accounts_json(MAC, DATETIME)
print("accounts_json:", accounts_json)

# 2) 키/IV 준비
key = derive_key_from_passphrase(PASSPHRASE)       # 32 bytes (AES-256)
iv = b"\x00" * 16                                 # 예시 IV (문서에 IV가 있으면 그걸 사용)

# 3) AES-256-CBC 암호화 (PKCS7 패딩 적용)
ciphertext = aes256_cbc_encrypt(accounts_json.encode("utf-8"), key, iv)
cipher = AES.new(key_bytes, AES.MODE_EBC)
    return cipher.encrypt(pad(plaintext_bytes, AES.block_size))

# 4) 암호문을 URL-safe base64로 인코딩 (padding 제거 선택 가능)
b64 = base64.urlsafe_b64encode(ciphertext).decode("ascii").rstrip("=")

# 5) URI 인코딩 (문서이행성 때문에 추가)
accounts_param = urllib.parse.quote_plus(b64)

In [120]:
text = "m05G3Thk9yD7+TLsb2VyS5SCg5JLVDB6/pggbVlkROEw1DkX1MdlevFM2LZzmuAeWrMKxzrpTg2Cuc2tpbILc8bBKKdXxUJhK09uM3vHCls="

decrypted_text = unpad(cipher.decrypt(text.encode('utf-8')), AES.block_size)

ValueError: Data must be aligned to block boundary in ECB mode

In [ ]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad
import base64

text = "m05G3Thk9yD7+TLsb2VyS5SCg5JLVDB6/pggbVlkROEw1DkX1MdlevFM2LZzmuAeWrMKxzrpTg2Cuc2tpbILc8bBKKdXxUJhK09uM3vHCls="
ciphertext = base64.b64decode(text)   # ← 중요: base64 디코딩

print("len(ciphertext) =", len(ciphertext), "mod 16 =", len(ciphertext) % 16)

key = b"your16bytekey!!"  # 16/24/32 바이트여야 함
cipher = AES.new(key, AES.MODE_ECB)

plaintext = unpad(cipher.decrypt(ciphertext), AES.block_size)
print(plaintext.decode('utf-8'))

In [ ]:
mac = (MAC_ADDRESS or "").strip().strip('"').strip("'").upper().replace(":", "-")
if not mac:
    raise SystemExit("MAC_ADDRESS가 비어있음")
print(mac)

# datetime 생성
dt = datetime.now().strftime('%Y%m%d%H%M%S')
print(dt)

# JSON 페이로드 생성
payload = {
    "mac_address": mac,
    "datetime": dt
}
plain_json = json.dumps(payload, separators=(',', ':'))

# AES 암호화 → Base64
aes = AESTestClass(plain_txt=plain_json, key=API_KEY)
b64_cipher = aes.encrypt()

# 인코딩 + 토큰 요청
endpoint = "https://apigateway.kisti.re.kr/tokenrequest.do"
params = {
    "accounts": b64_cipher,
    "client_id": CLIENT_ID
}

print("🔎 호출 URL 미리보기:", requests.Request('GET', endpoint, params=params).prepare().url)
response = requests.get(endpoint, params=params, timeout=10)
response.raise_for_status()
print("HTTP", response.status_code)

# 결과 출력
print(response.text)

data = response.json()
type(data)
token = data['access_token']
token
